### DeepL Translator




Remarks:

- Checking the counters, when does it make sense to check usage?
- Refactoring of the translator, outsourcing it to a separate method that translates the columns -->I will still do
- Insert Try Catch
- Mongo DB improrter einbauen

Explanation:
1. The program reads in the prepared keyword table.
2. Only then are the titles (the same happens with the abstracts) from one line
    2.1 Control which language is available
    2.2 If no English, then translate, entry in DF will be changed accordingly
3. Translate all keywords
4. Export data


In [ ]:
#Verbindung zu DeepL herstellen
import deepl

auth_key = "f63c02c5-f056-..."  # Replace with your key
translator = deepl.Translator(auth_key)


In [1]:
#Exchange import of data against Mongo DB reader
import pandas as pd

publication = pd.read_csv('wi_df_final_clean_keyword.csv',encoding='utf8')


In [2]:
publication.head()

,id,cfTitle,cfUri,keywords,doi,srcAuthors,authors,cfAbstr,publYear,eid,data_source,log,result_flag,error,error_doi,keyword_clean
0,10101316,Configuring Reference Models - An Integrated A...,NaN,"['Configuration', 'Decision support', 'Integra...",NaN,"Knackstedt R, Janiesch C, Rieke T","Knackstedt, Ralf;Janiesch, Christian;Rieke, To...","Reference models are of normative, universal n...",2006,2-s2.0-67649915340,scopus-api,{'log_title': 'configuring reference models - ...,True,NaN,NaN,"['Configuration, Decision support, Integration..."
1,10105507,Konnektionistische Tourenplanung für die dezen...,NaN,NaN,NaN,"Pietsch W, Teubner RA",NaN,Die Aufstellung effizienter Tourenpl&auml;ne s...,1992,NaN,NaN,NaN,False,NaN,NaN,NaN
2,10971967,Cryptocoin-Schulden – Haftung und Risikovertei...,NaN,NaN,NaN,Pesch,NaN,NaN,2017,NaN,NaN,NaN,False,NaN,NaN,NaN
3,10976734,Benefits and Perils of Virtual Modes of Organi...,NaN,NaN,10.4324/9781315601915,"Schellhammer Stefan, Klein Stefan","Schellhammer, Stefan;Klein, Stefan",NaN,2016,2-s2.0-85062635276,scopus-api,{'log_title': 'benefits and perils of virtual ...,True,NaN,NaN,NaN
4,10982470,Dynamic Decision Making in Energy Systems with...,NaN,"['Energy storage', 'Sequential decision making...",10.1007/978-3-319-51795-7_6,"Meisel S, Powell W","Meisel, Stephan;Powell, Warren B.",<html />,2017,2-s2.0-85048159112,scopus-api,{'log_title': 'dynamic decision making in ener...,True,NaN,NaN,"['Energy storage, Sequential decision making, ..."


In [ ]:
#Parameter von DeepL

def translate_titel_abstr(publication):
    """
    Funktion durchläuft alle Zeile des Pandas DF und schaut für die jeweiligen Einträge ob sie auf englisch vorleigen. Wenn dies nicht der Fall ist, werden die auf englisch übersetzt
    --> Datei wird nach jedem Übersetzungsvorgang exportiert, das hat den Grund das nicht ein Fehler auftritt und damit der Fortschritt umsost ist
    :param publication:
    :return:
    """
    #print(type(publication))
    publication_en = publication

    target_lang="EN"

    # Iterate over the rows of the DataFrame
    for index, row in publication_en.iterrows():

        actual_title = str(row['cfTitle'])
        actual_abstr = str(row['cfAbstr'])

        #When limit is reached, save
        actual_usage(publication_en)
        #translate titel
        if(actual_title != "" or type(actual_title)!= str):
            #checking the language
            if(check_language(actual_title) != "EN"):
                translated_titel = translator.translate_text(actual_abstr,target_lang=target_lang)
                #Insert translated value
                publication_en.loc[index, 'cfTitle'] = str(translated_titel)

        actual_usage(publication_en)
        #translate abstracts
        if(str(actual_abstr) != "" or type(actual_abstr)!= str):
            #checking the language
            if(check_language(actual_abstr) != "EN"):
                translated_abstr = translator.translate_text(actual_abstr,target_lang=target_lang)
                #Insert translated value
                publication_en.loc[index, 'cfAbstr'] = str(translated_abstr)


    return publication_en


def check_language(text):
    source_lang =text.detect_source_lang
    return source_lang
    pass


def csv_exporter(dataframe):
    dataframe.to_csv(r'.\wi_df_translated\wi_df_translated_DeepL_cached.csv', index=False)
    pass

def actual_usage(dataframe_to_save):
    '''
    Method checks if limit is reached, if so, cache
    :param dataframe_to_save:
    :return:
    '''
    usage = translator.get_usage()
    if usage.any_limit_reached:
        csv_exporter(dataframe_to_save)
    pass


#### Übersetzen der Keywords

In [ ]:
#### Function to translate Keywords
def translate_publication_keywords_wi(publication):
    """
    The function runs through all lines of the panda DF and looks for the respective entries to see whether they are in English. If this is not the case, they will be translated into English
    :param publication:
    :return:
    """
    publication_keywords = publication

    target_lang = 'EN'

    for index, row in publication_keywords.iterrows():
        data = row['keyword_clean']
        if(type(data) == str):
            #Convert a string containing a list to a list
            actual_keyword = eval(data)
            #for actual_str in actual_Data:  (If each element is to be translated individually)

            if(check_language(actual_keyword) != "EN"):
                actual_usage(publication_keywords)
                translated_keyword = translator.translate_text(actual_keyword,target_lang=target_lang)
                publication_keywords.loc[index, 'keywords'] = str(translated_keyword)

    return publication_keywords



In [ ]:
#Method to export when everything is translated

def csv_exporter(dataframe):
    dataframe.to_csv(r'.\wi_df_translated\wi_all_translated_final_DeepL.csv', index=False)
    pass


In [ ]:
#First, titles and abstracts are translated
translated_title_abstr = translate_titel_abstr(publication)
#Translate keywords
all_translated =translate_publication_keywords_wi(translated_title_abstr)
#Export a dataset with all entries translated to English
csv_exporter(all_translated)